### On créer un modèle depuis le fichier de la  pahse2 pour pouvoir associer  les 1 est 0 de la phase3 avec les valeur anomaly ou normal

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_score
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

dfPh2= pd.read_csv("files/Phase2.csv")
dfPh2 = dfPh2.fillna(dfPh2.median())
dfPh2= dfPh2.drop_duplicates(keep='first').reset_index()

txtDataColumn = ['protocol_type','service','flag']
dfPh2 = dfPh2.drop(txtDataColumn, axis= 1)

x = dfPh2.drop("class",axis = 1) #getting the features of the data in a matrix
y = dfPh2["class"]  # getting the label of the data in a  vector
X_train, X_test , y_train , y_test = train_test_split(x , y, test_size=0.3,stratify=y )

param_grid= {'n_neighbors' : [3,5,7,9,11,13,15,17], 
'metric' : ['manhattan', 'euclidean']
}
grid = GridSearchCV(KNeighborsClassifier(),param_grid, cv = 5) #initiamlisation du grid estimator 
grid.fit(X_train, y_train)# entrainement du grid (kfold)
print('les meilleur parametre pour notre knn sont : ', grid.best_params_)
modelKnn = grid.best_estimator_
print('score  ',round(modelKnn.score(X_test, y_test),3))







les meilleur parametre pour notre knn sont :  {'metric': 'manhattan', 'n_neighbors': 3}
score   0.938


In [2]:
df = pd.read_csv("files/fromPhase3.csv")
listemoinData = []
for i in range(20):
    temoindata = df.iloc[i]
    temoindata= temoindata.to_frame().transpose()
    classReal = modelKnn.predict(temoindata.drop("class",axis = 1)) 
    listemoinData.append([temoindata.iloc[0]['class'],classReal[0]])
print(listemoinData)

[[0.0, 'anomaly'], [0.0, 'anomaly'], [0.0, 'normal'], [0.0, 'anomaly'], [0.0, 'anomaly'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'anomaly'], [0.0, 'normal'], [0.0, 'anomaly'], [0.0, 'anomaly'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'normal'], [0.0, 'anomaly']]


D'après le résultat obtenue dans la dernier cellule en peut en conclure que , 0 = anomaly , 1 = normal 

### on remplace nos donner 

In [3]:
df["class"].replace(to_replace =[0,1], value = ['anomaly','normal'], inplace=True)

### faire un modéle knn avec les donner des deux phase 

In [4]:
df1 = pd.read_csv("files/Phase2.csv")
df3 = df1.append(df, ignore_index = True) 
####
df3 = df3.fillna(df3.median())
df3= df3.drop_duplicates(keep='first').reset_index()
txtDataColumn = ['protocol_type','service','flag']
df3 = pd.get_dummies(df3,columns=txtDataColumn) 

In [5]:
x = df3.drop("class",axis = 1) #getting the features of the data in a matrix 
y = df3["class"] #getting the labes of the data in a vector 
X_train, X_test , y_train , y_test = train_test_split(x , y, test_size=0.3,stratify=y ) 

param_grid= {'n_neighbors' : [3,5,7,9,11,13,15,17], 
'metric' : ['manhattan', 'euclidean']
}
grid = GridSearchCV(KNeighborsClassifier(),param_grid, cv = 5) #initiamlisation du grid estimator 

grid.fit(X_train, y_train)

print('les meilleur parametre pour notre knn sont : ', grid.best_params_)
print('score de notre modelle : ' , round(grid.best_score_,3))

bestEstimator = grid.best_estimator_#sauvgarde du modelle 



les meilleur parametre pour notre knn sont :  {'metric': 'manhattan', 'n_neighbors': 3}
score de notre modelle :  0.968


### on obtient ainsi un classificateur robuste avec , avec un score de 0.968
